<a href="https://colab.research.google.com/github/ykato27/AutoKeras/blob/main/AutoKeras_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autokeras

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import tensorflow as tf
import autokeras as ak

In [ ]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

df_train = pd.concat([x_train, y_train], axis=1)
df_test = pd.concat([x_test, y_test], axis=1)

feature_names = boston.feature_names
label_name = ["MEDV"]

In [ ]:
reg = ak.StructuredDataRegressor(
    overwrite=True,
    max_trials=10,  # It tries 10 different models.
    )

In [ ]:
reg.fit(
    x=df_train[feature_names],
    y=df_train[label_name],
    validation_split=0.2,
    )

Trial 10 Complete [00h 00m 10s]
val_loss: 26.017635345458984

Best val_loss So Far: 14.743266105651855
Total elapsed time: 00h 05m 31s
INFO:tensorflow:Oracle triggered exit
Epoch 1/37
13/13 [==============================] - 0s 2ms/step - loss: 185.5670 - mean_squared_error: 185.5670
Epoch 2/37
13/13 [==============================] - 0s 2ms/step - loss: 44.2478 - mean_squared_error: 44.2478
Epoch 3/37
13/13 [==============================] - 0s 2ms/step - loss: 25.5493 - mean_squared_error: 25.5493
Epoch 4/37
13/13 [==============================] - 0s 2ms/step - loss: 17.5457 - mean_squared_error: 17.5457
Epoch 5/37
13/13 [==============================] - 0s 2ms/step - loss: 13.9778 - mean_squared_error: 13.9778
Epoch 6/37
13/13 [==============================] - 0s 2ms/step - loss: 12.3760 - mean_squared_error: 12.3760
Epoch 7/37
13/13 [==============================] - 0s 2ms/step - loss: 11.9614 - mean_squared_error: 11.9614
Epoch 8/37
13/13 [==============================] - 0s 

In [ ]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 13)                0         
_________________________________________________________________
normalization (Normalization (None, 13)                27        
_________________________________________________________________
dense (Dense)                (None, 1024)              14336     
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                32800     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [ ]:
y_pred = reg.predict(x=df_test[feature_names])

4/4 [==============================] - 0s 3ms/step


In [ ]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [ ]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.876561  2.873366  12.909305  3.592952
